# DistFlow 单点潮流计算
# DistFlow Single Point Power Flow Calculation

基于论文中式(13)-(20)的DistFlow+锥松弛模型实现配电网潮流计算

## 数学模型

### 有功功率平衡 (13)
$$\sum_{j \in \delta(i)} P_{ij} - \sum_{j \in \gamma(i)} (P_{ji} - R_{ji}I_{ji}^{sqr}) = P_i^{DG} - P_{i,max}^L + P_i^{L,shd}$$

### 无功功率平衡 (14)
$$\sum_{j \in \delta(i)} Q_{ij} - \sum_{j \in \gamma(i)} (Q_{ji} - X_{ji}I_{ji}^{sqr}) = Q_i^{DG} - Q_{i,max}^L + Q_i^{L,shd}$$

### 电压约束 (15)-(16)
锥松弛形式的电压降约束

### 电压限制 (17)
$$V_{i,min}^{sqr} \leq V_i^{sqr} \leq V_{i,max}^{sqr}$$

### 电流限制 (18)
$$0 \leq I_{ij}^{sqr} \leq \alpha_{ij} I_{ij,max}^{sqr}$$

### 功率约束 (19)
$$P_{ij}^2 + Q_{ij}^2 \leq V_i^{sqr} I_{ij}^{sqr}$$

### 二阶锥约束 (20)
$$\left\|\begin{pmatrix} 2P_{ij} \\ 2Q_{ij} \\ I_{ij}^{sqr} - V_i^{sqr} \end{pmatrix}\right\|_2 \leq I_{ij}^{sqr} + V_i^{sqr}$$

In [1]:
# 导入必要的库
import numpy as np
import pandas as pd
import cvxpy as cp
import matplotlib.pyplot as plt
import sys
import os

# 添加项目路径
sys.path.append('..')
from src.datasets.loader import load_system_data

print("库导入成功！")

库导入成功！


In [2]:
# 加载系统数据
try:
    system_data = load_system_data('../data')
    print("数据加载成功！")
    print(f"发电机数量: {len(system_data.generators)}")
    print(f"负荷节点数量: {len(system_data.loads)}")
    print(f"支路数量: {len(system_data.branches)}")
    print(f"系统额定电压: {system_data.system_parameters.nominal_voltage} kV")
except Exception as e:
    print(f"数据加载失败: {e}")
    # 如果数据加载失败，我们使用IEEE 33节点标准数据
    print("将使用IEEE 33节点标准配电网数据...")

警告：33节点系统通常有32条支路，但当前有2条支路
警告：以下节点没有支路连接: {4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33}
数据加载成功！
发电机数量: 14
负荷节点数量: 33
支路数量: 2
系统额定电压: 12.66 kV


In [3]:
# IEEE 33节点标准配电网数据 (用于支路参数)
# IEEE 33-bus standard distribution network data

# 支路数据: [起始节点, 终止节点, 电阻(Ω), 电抗(Ω)]
branch_data = np.array([
    [1, 2, 0.0922, 0.0470],
    [2, 3, 0.4930, 0.2511],
    [3, 4, 0.3660, 0.1864],
    [4, 5, 0.3811, 0.1941],
    [5, 6, 0.8190, 0.7070],
    [6, 7, 0.1872, 0.6188],
    [7, 8, 0.7114, 0.2351],
    [8, 9, 1.0300, 0.7400],
    [9, 10, 1.0440, 0.7400],
    [10, 11, 0.1966, 0.0650],
    [11, 12, 0.3744, 0.1238],
    [12, 13, 1.4680, 1.1550],
    [13, 14, 0.5416, 0.7129],
    [14, 15, 0.5910, 0.5260],
    [15, 16, 0.7463, 0.5450],
    [16, 17, 1.2890, 1.7210],
    [17, 18, 0.7320, 0.5740],
    [2, 19, 0.1640, 0.1565],
    [19, 20, 1.5042, 1.3554],
    [20, 21, 0.4095, 0.4784],
    [21, 22, 0.7089, 0.9373],
    [3, 23, 0.4512, 0.3083],
    [23, 24, 0.8980, 0.7091],
    [24, 25, 0.8960, 0.7011],
    [6, 26, 0.2030, 0.1034],
    [26, 27, 0.2842, 0.1447],
    [27, 28, 1.0590, 0.9337],
    [28, 29, 0.8042, 0.7006],
    [29, 30, 0.5075, 0.2585],
    [30, 31, 0.9744, 0.9630],
    [31, 32, 0.3105, 0.3619],
    [32, 33, 0.3410, 0.5302]
])

# 负荷数据: [节点, 有功功率(kW), 无功功率(kvar)] 
load_data = np.array([
    [1, 0, 0], [2, 100, 60], [3, 90, 40], [4, 120, 80], [5, 60, 30],
    [6, 60, 20], [7, 200, 100], [8, 200, 100], [9, 60, 20], [10, 60, 20],
    [11, 45, 30], [12, 60, 35], [13, 60, 35], [14, 120, 80], [15, 60, 10],
    [16, 60, 20], [17, 60, 20], [18, 90, 40], [19, 90, 40], [20, 90, 40],
    [21, 90, 40], [22, 90, 40], [23, 90, 50], [24, 420, 200], [25, 420, 200],
    [26, 60, 25], [27, 60, 25], [28, 40, 20], [29, 120, 70], [30, 200, 600],
    [31, 150, 70], [32, 210, 100], [33, 60, 40]
])

# 系统参数
V_base = 12.66  # kV 基准电压
V_min = 11.39   # kV 最小电压
V_max = 13.92   # kV 最大电压
S_base = 1000   # kVA 基准功率

print(f"支路数量: {len(branch_data)}")
print(f"负荷节点数量: {len(load_data)}")
print(f"基准电压: {V_base} kV")

支路数量: 32
负荷节点数量: 33
基准电压: 12.66 kV


In [4]:
class DistFlowSolver:
    """DistFlow潮流求解器"""
    
    def __init__(self, branch_data, load_data, system_params):
        self.branch_data = branch_data
        self.load_data = load_data
        self.V_base = system_params['V_base']
        self.V_min = system_params['V_min']
        self.V_max = system_params['V_max']
        
        # 网络拓扑
        self.n_buses = int(max(branch_data[:, 1]))  # 节点数
        self.n_branches = len(branch_data)  # 支路数
        
        # 构建邻接关系
        self.children = {i: [] for i in range(1, self.n_buses + 1)}
        self.parent = {i: None for i in range(1, self.n_buses + 1)}
        
        for i, (from_bus, to_bus, r, x) in enumerate(branch_data):
            from_bus, to_bus = int(from_bus), int(to_bus)
            self.children[from_bus].append(to_bus)
            self.parent[to_bus] = from_bus
        
        print(f"网络构建完成: {self.n_buses}个节点, {self.n_branches}条支路")
    
    def solve_power_flow(self, with_cone_relaxation=True):
        """求解DistFlow潮流"""
        
        # 决策变量
        # 电压平方 V_i^2
        V_sqr = cp.Variable(self.n_buses + 1, name="V_sqr")
        
        # 有功和无功功率流 P_ij, Q_ij
        P = cp.Variable((self.n_buses + 1, self.n_buses + 1), name="P")
        Q = cp.Variable((self.n_buses + 1, self.n_buses + 1), name="Q")
        
        # 电流平方 I_ij^2
        I_sqr = cp.Variable((self.n_buses + 1, self.n_buses + 1), name="I_sqr")
        
        # 约束条件
        constraints = []
        
        # 1. 根节点电压约束 (节点1为根节点)
        constraints.append(V_sqr[1] == self.V_base**2)
        
        # 2. 电压限制约束 (17)
        for i in range(1, self.n_buses + 1):
            constraints.append(V_sqr[i] >= self.V_min**2)
            constraints.append(V_sqr[i] <= self.V_max**2)
        
        # 3. 遍历每条支路
        for from_bus, to_bus, r, x in self.branch_data:
            from_bus, to_bus = int(from_bus), int(to_bus)
            
            # 电压降约束 (15)-(16)
            # V_j^2 = V_i^2 - 2(R_ij*P_ij + X_ij*Q_ij) + (R_ij^2 + X_ij^2)*I_ij^2
            voltage_drop = V_sqr[from_bus] - 2*(r*P[from_bus, to_bus] + x*Q[from_bus, to_bus]) + (r**2 + x**2)*I_sqr[from_bus, to_bus]
            
            if with_cone_relaxation:
                # 锥松弛: V_j^2 <= voltage_drop (15)
                constraints.append(V_sqr[to_bus] <= voltage_drop)
                # 另一个方向的约束 (16)
                constraints.append(V_sqr[to_bus] >= voltage_drop - 1000*(1 - 1))  # 这里简化了α_ij项
            else:
                # 严格相等约束
                constraints.append(V_sqr[to_bus] == voltage_drop)
            
            # 电流限制约束 (18) - 假设足够大的电流限制
            I_max_sqr = 1000  # 临时设定较大值
            constraints.append(I_sqr[from_bus, to_bus] >= 0)
            constraints.append(I_sqr[from_bus, to_bus] <= I_max_sqr)
            
            # 功率约束 (19): P_ij^2 + Q_ij^2 <= V_i^2 * I_ij^2
            constraints.append(P[from_bus, to_bus]**2 + Q[from_bus, to_bus]**2 <= V_sqr[from_bus] * I_sqr[from_bus, to_bus])
            
            # 二阶锥约束 (20)
            if with_cone_relaxation:
                soc_constraint = cp.SOC(
                    I_sqr[from_bus, to_bus] + V_sqr[from_bus],
                    cp.hstack([2*P[from_bus, to_bus], 2*Q[from_bus, to_bus], I_sqr[from_bus, to_bus] - V_sqr[from_bus]])
                )
                constraints.append(soc_constraint)
        
        # 4. 功率平衡约束 (13)-(14)
        for i in range(1, self.n_buses + 1):
            # 获取节点i的负荷
            load_idx = np.where(self.load_data[:, 0] == i)[0]
            if len(load_idx) > 0:
                P_load = self.load_data[load_idx[0], 1]
                Q_load = self.load_data[load_idx[0], 2]
            else:
                P_load = Q_load = 0
            
            # 有功功率平衡 (13)
            P_injection = 0  # 暂不考虑分布式发电
            P_outflow = cp.sum([P[i, j] for j in self.children[i]])
            P_inflow = P[self.parent[i], i] - r * I_sqr[self.parent[i], i] if self.parent[i] is not None else 0
            
            if i == 1:  # 根节点
                constraints.append(P_outflow == cp.sum([self.load_data[k, 1] for k in range(len(self.load_data))]))
            else:
                # 找到对应的支路电阻
                parent_branch = np.where((self.branch_data[:, 0] == self.parent[i]) & (self.branch_data[:, 1] == i))[0]
                if len(parent_branch) > 0:
                    r_parent = self.branch_data[parent_branch[0], 2]
                    constraints.append(P_inflow - P_outflow == P_load)
            
            # 无功功率平衡 (14) - 类似处理
            Q_injection = 0
            Q_outflow = cp.sum([Q[i, j] for j in self.children[i]])
            if i != 1 and self.parent[i] is not None:
                parent_branch = np.where((self.branch_data[:, 0] == self.parent[i]) & (self.branch_data[:, 1] == i))[0]
                if len(parent_branch) > 0:
                    x_parent = self.branch_data[parent_branch[0], 3]
                    Q_inflow = Q[self.parent[i], i] - x_parent * I_sqr[self.parent[i], i]
                    constraints.append(Q_inflow - Q_outflow == Q_load)
        
        # 目标函数：最小化系统损耗
        total_losses = 0
        for from_bus, to_bus, r, x in self.branch_data:
            from_bus, to_bus = int(from_bus), int(to_bus)
            total_losses += r * I_sqr[from_bus, to_bus]
        
        objective = cp.Minimize(total_losses)
        
        # 求解问题
        problem = cp.Problem(objective, constraints)
        
        try:
            problem.solve(solver=cp.CLARABEL, verbose=True)
            
            if problem.status == cp.OPTIMAL:
                print(f"\n求解成功! 状态: {problem.status}")
                print(f"目标值 (总损耗): {problem.value:.6f} kW")
                
                # 提取结果
                voltages = np.sqrt(V_sqr.value[1:self.n_buses+1])  # 电压幅值
                
                return {
                    'status': problem.status,
                    'objective': problem.value,
                    'voltages': voltages,
                    'V_sqr': V_sqr.value,
                    'P': P.value,
                    'Q': Q.value,
                    'I_sqr': I_sqr.value
                }
            else:
                print(f"求解失败! 状态: {problem.status}")
                return None
                
        except Exception as e:
            print(f"求解过程中出错: {e}")
            return None

print("DistFlow求解器类定义完成")

DistFlow求解器类定义完成


In [5]:
# 创建求解器实例并求解
system_params = {
    'V_base': V_base,
    'V_min': V_min,
    'V_max': V_max
}

solver = DistFlowSolver(branch_data, load_data, system_params)

print("开始求解DistFlow潮流...")
result = solver.solve_power_flow(with_cone_relaxation=True)

if result:
    print("\n=== 潮流计算结果 ===")
    print(f"求解状态: {result['status']}")
    print(f"系统总损耗: {result['objective']:.6f} kW")
    
    # 显示各节点电压
    print("\n各节点电压 (kV):")
    for i, voltage in enumerate(result['voltages'], 1):
        print(f"节点 {i:2d}: {voltage:.4f} kV")
    
    # 检查电压是否在合理范围内
    print(f"\n电压范围检查:")
    print(f"最低电压: {min(result['voltages']):.4f} kV (应 >= {V_min:.2f} kV)")
    print(f"最高电压: {max(result['voltages']):.4f} kV (应 <= {V_max:.2f} kV)")
    
    if min(result['voltages']) >= V_min and max(result['voltages']) <= V_max:
        print("✓ 所有节点电压都在允许范围内")
    else:
        print("✗ 部分节点电压超出允许范围")
else:
    print("潮流求解失败")

(CVXPY) Jun 22 09:22:41 PM: Your problem has 3502 variables, 324 constraints, and 0 parameters.
(CVXPY) Jun 22 09:22:41 PM: It is compliant with the following grammars: 
(CVXPY) Jun 22 09:22:41 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jun 22 09:22:41 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jun 22 09:22:41 PM: Your problem is compiled with the CPP canonicalization backend.


网络构建完成: 33个节点, 32条支路
开始求解DistFlow潮流...
                                     CVXPY                                     
                                     v1.6.6                                    
求解过程中出错: Problem does not follow DCP rules. Specifically:
The following constraints are not DCP:
power(P[1, 2], 2.0) + power(Q[1, 2], 2.0) <= V_sqr[1] @ I_sqr[1, 2] , because the following subexpressions are not:
|--  V_sqr[1] @ I_sqr[1, 2]
power(P[2, 3], 2.0) + power(Q[2, 3], 2.0) <= V_sqr[2] @ I_sqr[2, 3] , because the following subexpressions are not:
|--  V_sqr[2] @ I_sqr[2, 3]
power(P[3, 4], 2.0) + power(Q[3, 4], 2.0) <= V_sqr[3] @ I_sqr[3, 4] , because the following subexpressions are not:
|--  V_sqr[3] @ I_sqr[3, 4]
power(P[4, 5], 2.0) + power(Q[4, 5], 2.0) <= V_sqr[4] @ I_sqr[4, 5] , because the following subexpressions are not:
|--  V_sqr[4] @ I_sqr[4, 5]
power(P[5, 6], 2.0) + power(Q[5, 6], 2.0) <= V_sqr[5] @ I_sqr[5, 6] , because the following subexpressions are not:
|--  V_sqr[5

In [ ]:
# 可视化结果
if result:
    plt.figure(figsize=(12, 8))
    
    # 子图1: 各节点电压
    plt.subplot(2, 2, 1)
    bus_numbers = range(1, len(result['voltages']) + 1)
    plt.plot(bus_numbers, result['voltages'], 'bo-', markersize=4)
    plt.axhline(y=V_min, color='r', linestyle='--', label=f'最低电压 {V_min} kV', alpha=0.7)
    plt.axhline(y=V_max, color='r', linestyle='--', label=f'最高电压 {V_max} kV', alpha=0.7)
    plt.axhline(y=V_base, color='g', linestyle='-', label=f'额定电压 {V_base} kV', alpha=0.7)
    plt.xlabel('节点编号')
    plt.ylabel('电压 (kV)')
    plt.title('各节点电压分布')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 子图2: 电压偏差
    plt.subplot(2, 2, 2)
    voltage_deviation = (result['voltages'] - V_base) / V_base * 100
    plt.plot(bus_numbers, voltage_deviation, 'ro-', markersize=4)
    plt.xlabel('节点编号')
    plt.ylabel('电压偏差 (%)')
    plt.title('各节点电压偏差')
    plt.grid(True, alpha=0.3)
    
    # 子图3: 负荷分布
    plt.subplot(2, 2, 3)
    plt.bar(load_data[:, 0], load_data[:, 1], alpha=0.7, label='有功负荷')
    plt.bar(load_data[:, 0], load_data[:, 2], alpha=0.7, label='无功负荷')
    plt.xlabel('节点编号')
    plt.ylabel('负荷 (kW/kvar)')
    plt.title('各节点负荷分布')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 子图4: 系统概览
    plt.subplot(2, 2, 4)
    summary_data = [
        ['节点数', len(result['voltages'])],
        ['支路数', len(branch_data)],
        ['总负荷(kW)', sum(load_data[:, 1])],
        ['总损耗(kW)', result['objective']],
        ['损耗率(%)', result['objective']/sum(load_data[:, 1])*100]
    ]
    
    plt.axis('off')
    table = plt.table(cellText=[[item[0], f'{item[1]:.3f}'] for item in summary_data],
                     colLabels=['参数', '数值'],
                     cellLoc='center',
                     loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 1.5)
    plt.title('系统概览')
    
    plt.tight_layout()
    plt.show()
    
    print("\n=== 结果验证 ===")
    print(f"电压是否在预期范围 ({V_min:.2f}-{V_max:.2f} kV): ", end="")
    if min(result['voltages']) >= V_min and max(result['voltages']) <= V_max:
        print("✓ 是")
    else:
        print("✗ 否")
    
    print(f"系统损耗率: {result['objective']/sum(load_data[:, 1])*100:.3f}%")
    print(f"最大电压偏差: {max(abs(voltage_deviation)):.3f}%")
    
else:
    print("没有结果可供可视化")

没有结果可供可视化


: 